# Based on Drabas & Lee  -- Learning PySpark
## Resilient Distributed Datasets
### DataFrames
#### Start the jupyter notebook from its own folder, otherwise python might not find some files to load!
set the kernel to python 2 or Python [default]!

This notebook contains sample code from [Learning PySpark]() focusing on PySpark and DataFrames.

A **DataFrame** is an immutable distributed collection of data that is organized into 
named columns analogous to a table in a relational database. 

A Spark DataFrame is a similar to Pandas DataFrame.

Using standard RDDs can be slow due to communication overhead between the Java JVM and Py4J. DataFrame has a special structure and they can be faster.

Whenever a PySpark program is executed using RDDs, there is a potentially large overhead to execute the job. 

The Spark Context uses Py4j to launch a JVM using the JavaSparkContext. 

Any RDD transformations are initially mapped to PythonRDD objects in Java.
Once these tasks are pushed out to the Spark Worker(s), PythonRDD objects launch 
Python subprocesses using pipes to send both code and data to be processed within 
Python. Python can take advantage of the performance optimizations in Spark (even while 
the codebase for the Catalyst Optimizer is written in Scala). Basically, it is a Python 
wrapper of approximately 2,000 lines of code that allows PySpark DataFrame queries 
to be significantly faster.

### Generate your own DataFrame
Instead of accessing the file system, let's create a DataFrame by generating the data.  In this case, we'll first create the `stringRDD` RDD and then convert it into a DataFrame when we're reading `stringJSONRDD` using `spark.read.json`.

In [1]:
#import findspark
#findspark.init()
#import pyspark

#from pyspark.context import SparkContext
#from pyspark.sql.session import SparkSession
#sc_local = SparkContext('local')
#spark = SparkSession(sc_local)

In [2]:
sc

<SparkContext master=spark://ec2-18-223-209-87.us-east-2.compute.amazonaws.com:7077 appName=PySparkShell>

In [3]:
# Generate our own JSON data 
#   This way we don't have to access the file system yet.
stringJSONRDD = sc.parallelize((""" 
  { "id": "123",
    "name": "Katie",
    "age": 19,
    "eyeColor": "brown"
  }""",
   """{
    "id": "234",
    "name": "Michael",
    "age": 22,
    "eyeColor": "green"
  }""", 
  """{
    "id": "345",
    "name": "Simone",
    "age": 23,
    "eyeColor": "blue"
  }""")
)

We could also use Hadoop Distributed File System (HDFS), or other cloud storage systems (for 
example, S3 or WASB) to load data and create Dataframes.

Now that we have created the RDD, we will convert this into a DataFrame by using 
the SparkSession read.json method (that is, spark.read.json(...)). We will also 
create a temporary table by using the .createOrReplaceTempView method.

In [4]:
# Create DataFrame from json RDD
swimmersJSON = spark.read.json(stringJSONRDD)

In [5]:
# Create temporary table
swimmersJSON.createOrReplaceTempView("swimmersJSON")

In [6]:
# DataFrame API
swimmersJSON.show()

+---+--------+---+-------+
|age|eyeColor| id|   name|
+---+--------+---+-------+
| 19|   brown|123|  Katie|
| 22|   green|234|Michael|
| 23|    blue|345| Simone|
+---+--------+---+-------+



We can run **SQL queries** on the dataframes objects:

In [9]:
# SQL Query
spark.sql("select * from swimmersJSON").collect()

[Row(age=19, eyeColor=u'brown', id=u'123', name=u'Katie'),
 Row(age=22, eyeColor=u'green', id=u'234', name=u'Michael'),
 Row(age=23, eyeColor=u'blue', id=u'345', name=u'Simone')]

#### Inferring the Schema Using Reflection
Note that Apache Spark is inferring the schema using reflection; i.e. it automaticlaly determines the schema of the data based on reviewing the JSON data.

In [10]:
# Print the schema
swimmersJSON.printSchema()

root
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)



Notice that Spark was able to infer the schema (using `.printSchema`).

We can also programmatically specify the schema.

#### Programmatically Specifying the Schema
In this case, let's specify the schema for a `CSV` text file.

In [14]:
from pyspark.sql.types import *

# Generate our own CSV data 
#   This way we don't have to access the file system yet.
stringCSVRDD = sc.parallelize([(123, 'Katie', 19, 'brown'), (234, 'Michael', 22, 'green'), (345, 'Simone', 23, 'blue')])

# The schema is encoded in a string, using StructType we define the schema using various pyspark.sql.types
#schemaString = "id name age eyeColor"
schema = StructType([
    StructField("id", LongType(), True),    
    StructField("name", StringType(), True),
    StructField("age", LongType(), True),
    StructField("eyeColor", StringType(), True)
])

In [15]:
# Apply the schema to the RDD and Create DataFrame
swimmers = spark.createDataFrame(stringCSVRDD, schema)

# Creates a temporary view using the DataFrame
swimmers.createOrReplaceTempView("swimmers")

In [16]:
# Print the schema
#   Notice that we have redefined id as Long (instead of String)
swimmers.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)



Note, the StructField class is broken down in terms of: <br>
* name: The name of this field 
* dataType: The data type of this field
* nullable: Indicates whether values of this field can be null

Additional Resources include:
* [PySpark API Reference](https://spark.apache.org/docs/2.0.0/api/python/pyspark.sql.html)
* [Spark SQL, DataFrames, and Datasets Guide](https://spark.apache.org/docs/latest/sql-programming-guide.html#programmatically-specifying-the-schema): This is in reference to Programmatically Specifying the Schema using a `CSV` file.

### Number of rows
To get the number of rows within your DataFrame, you can use the count() method:

In [18]:
swimmers.count()

3

In [26]:
# Same using SQL. Get count of rows in SQL:
spark.sql("select count(1) from swimmers").show()

+--------+
|count(1)|
+--------+
|       3|
+--------+



#### Running filter statements
To run a filter statement, you can use the filter clause; in the following code 
snippet, we are using the select clause to specify the columns to be returned  
as well:


In [27]:
# Query id and age for swimmers with age = 22 via DataFrame API
swimmers.select("id", "age").filter("age = 22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [29]:
# Another way to write the above query is below
swimmers.select(swimmers.id, swimmers.age).filter(swimmers.age == 22).show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [30]:
# Same with SQL. Query id and age for swimmers with age = 22 in SQL
spark.sql("select id, age from swimmers where age = 22").show()


+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [33]:
# Show the values 
swimmers.show()

+---+-------+---+--------+
| id|   name|age|eyeColor|
+---+-------+---+--------+
|123|  Katie| 19|   brown|
|234|Michael| 22|   green|
|345| Simone| 23|    blue|
+---+-------+---+--------+



In [32]:
#We can do sql queries in dataframes: Execute SQL Query and return the data
spark.sql("select * from swimmers").show()

+---+-------+---+--------+
| id|   name|age|eyeColor|
+---+-------+---+--------+
|123|  Katie| 19|   brown|
|234|Michael| 22|   green|
|345| Simone| 23|    blue|
+---+-------+---+--------+



If we only want to get back the name of the swimmers who have an eye colorthat begins with the letter b, we can use a SQL-like syntax, like, as shown in the following code:

In [22]:
# Get the name, eyeColor where eyeColor like 'b%'
swimmers.select("name", "eyeColor").filter("eyeColor like 'b%'").show()

+------+--------+
|  name|eyeColor|
+------+--------+
| Katie|   brown|
|Simone|    blue|
+------+--------+



In [31]:
#Same qith SQL.  Query name and eye color for swimmers with eye color starting with the letter 'b'
spark.sql("select name, eyeColor from swimmers where eyeColor like 'b%'").show()

+------+--------+
|  name|eyeColor|
+------+--------+
| Katie|   brown|
|Simone|    blue|
+------+--------+



## On-Time Flight Performance
Query flight departure delays by State and City by joining the departure delay and join to the airport codes (to identify state and city).

### DataFrame Queries
Let's run a flight performance using DataFrames; let's first build the DataFrames from the source datasets.

In [34]:
#hdfs dfs -mkdir -p /hdfs_data
!hdfs dfs -ls /


Found 3 items
drwxr-xr-x   - ec2-user supergroup          0 2019-02-10 23:57 /hdfs_data
drwx-wx-wx   - ec2-user supergroup          0 2019-02-11 01:54 /tmp
drwxr-xr-x   - ec2-user supergroup          0 2019-02-06 10:32 /user


In [37]:
# uncomment if the filed doesn't exist in hdfs
#!hdfs dfs -put data/flight-data/departuredelays.csv /hdfs_data
!hdfs fsck /hdfs_data/departuredelays.csv


Connecting to namenode via http://ec2-18-223-209-87.us-east-2.compute.amazonaws.com:50070/fsck?ugi=ec2-user&path=%2Fhdfs_data%2Fdeparturedelays.csv
FSCK started by ec2-user (auth:SIMPLE) from /172.31.5.183 for path /hdfs_data/departuredelays.csv at Mon Feb 11 02:17:27 UTC 2019
.
/hdfs_data/departuredelays.csv:  Under replicated BP-663532545-172.31.27.125-1549216637007:blk_1073741835_1011. Target Replicas is 3 but found 2 live replica(s), 0 decommissioned replica(s) and 0 decommissioning replica(s).
Status: HEALTHY
 Total size:	33396236 B
 Total dirs:	0
 Total files:	1
 Total symlinks:		0
 Total blocks (validated):	1 (avg. block size 33396236 B)
 Minimally replicated blocks:	1 (100.0 %)
 Over-replicated blocks:	0 (0.0 %)
 Under-replicated blocks:	1 (100.0 %)
 Mis-replicated blocks:		0 (0.0 %)
 Default replication factor:	3
 Average block replication:	2.0
 Corrupt blocks:		0
 Missing replicas:		1 (33.333332 %)
 Number of data-nodes:		2
 Number of racks:		1
FSCK ended at Mon Feb 11 02:17:

In [39]:
#uncomment if the file doesn't exisit in hdfs
#!hdfs dfs -put data/flight-data/airport-codes-na.txt /hdfs_data
!hdfs fsck /hdfs_data/airport-codes-na.txt



Connecting to namenode via http://ec2-18-223-209-87.us-east-2.compute.amazonaws.com:50070/fsck?ugi=ec2-user&path=%2Fhdfs_data%2Fairport-codes-na.txt
FSCK started by ec2-user (auth:SIMPLE) from /172.31.5.183 for path /hdfs_data/airport-codes-na.txt at Mon Feb 11 02:18:02 UTC 2019
.
/hdfs_data/airport-codes-na.txt:  Under replicated BP-663532545-172.31.27.125-1549216637007:blk_1073741836_1012. Target Replicas is 3 but found 2 live replica(s), 0 decommissioned replica(s) and 0 decommissioning replica(s).
Status: HEALTHY
 Total size:	11411 B
 Total dirs:	0
 Total files:	1
 Total symlinks:		0
 Total blocks (validated):	1 (avg. block size 11411 B)
 Minimally replicated blocks:	1 (100.0 %)
 Over-replicated blocks:	0 (0.0 %)
 Under-replicated blocks:	1 (100.0 %)
 Mis-replicated blocks:		0 (0.0 %)
 Default replication factor:	3
 Average block replication:	2.0
 Corrupt blocks:		0
 Missing replicas:		1 (33.333332 %)
 Number of data-nodes:		2
 Number of racks:		1
FSCK ended at Mon Feb 11 02:18:02 

In [41]:
# Set File Paths
flightPerfFilePath = "/hdfs_data/departuredelays.csv"
airportsFilePath = "/hdfs_data/airport-codes-na.txt"


### Load dataset from a  file and show it

In [42]:
# Obtain Airports dataset
airports = spark.read.csv(airportsFilePath, header='true', inferSchema='true', sep='\t')
airports.createOrReplaceTempView("airports")
airports.show()

+-----------+-----+-------+----+
|       City|State|Country|IATA|
+-----------+-----+-------+----+
| Abbotsford|   BC| Canada| YXX|
|   Aberdeen|   SD|    USA| ABR|
|    Abilene|   TX|    USA| ABI|
|      Akron|   OH|    USA| CAK|
|    Alamosa|   CO|    USA| ALS|
|     Albany|   GA|    USA| ABY|
|     Albany|   NY|    USA| ALB|
|Albuquerque|   NM|    USA| ABQ|
| Alexandria|   LA|    USA| AEX|
|  Allentown|   PA|    USA| ABE|
|   Alliance|   NE|    USA| AIA|
|     Alpena|   MI|    USA| APN|
|    Altoona|   PA|    USA| AOO|
|   Amarillo|   TX|    USA| AMA|
|Anahim Lake|   BC| Canada| YAA|
|  Anchorage|   AK|    USA| ANC|
|   Appleton|   WI|    USA| ATW|
|     Arviat|  NWT| Canada| YEK|
|  Asheville|   NC|    USA| AVL|
|      Aspen|   CO|    USA| ASE|
+-----------+-----+-------+----+
only showing top 20 rows



#### load another dataset and show it

In [43]:
# Obtain Departure Delays dataset
flightPerf = spark.read.csv(flightPerfFilePath, header='true')
flightPerf.createOrReplaceTempView("FlightPerformance")
flightPerf.show()


+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
|01030605|    0|     602|   ABE|        ATL|
|01041243|   10|     602|   ABE|        ATL|
|01040605|   28|     602|   ABE|        ATL|
|01051245|   88|     602|   ABE|        ATL|
|01050605|    9|     602|   ABE|        ATL|
|01061215|   -6|     602|   ABE|        ATL|
|01061725|   69|     602|   ABE|        ATL|
|01061230|    0|     369|   ABE|        DTW|
|01060625|   -3|     602|   ABE|        ATL|
|01070600|    0|     369|   ABE|        DTW|
|01071725|    0|     602|   ABE|        ATL|
|01071230|    0|     369|   ABE|        DTW|
|01070625|    0|     602|   ABE|        ATL|
|01071219|    0|     569|   ABE|        ORD|
|01080600|

In [44]:
# Cache the Departure Delays dataset. This way subsequent quieries can be faster.
flightPerf.cache()

DataFrame[date: string, delay: string, distance: string, origin: string, destination: string]

In [47]:
# Query Sum of Flight Delays by City and Origin Code (for Washington State)
spark.sql("select a.City, \
          f.origin, \
          sum(f.delay) as Delays \
          from FlightPerformance f \
          join airports a \
          on a.IATA = f.origin \
          where a.State = 'WA' \
          group by a.City, f.origin \
          order by sum(f.delay) desc").show()

+-------+------+--------+
|   City|origin|  Delays|
+-------+------+--------+
|Seattle|   SEA|159086.0|
|Spokane|   GEG| 12404.0|
|  Pasco|   PSC|   949.0|
+-------+------+--------+



In [48]:
# Query Sum of Flight Delays by State (for the US)
spark.sql("select a.State, \
          sum(f.delay) as Delays \
          from FlightPerformance f \
          join airports a \
          on a.IATA = f.origin \
          where a.Country = 'USA' \
          group by a.State ").show()

+-----+---------+
|State|   Delays|
+-----+---------+
|   SC|  80666.0|
|   AZ| 401793.0|
|   LA| 199136.0|
|   MN| 256811.0|
|   NJ| 452791.0|
|   OR| 109333.0|
|   VA|  98016.0|
| null| 397237.0|
|   RI|  30760.0|
|   WY|  15365.0|
|   KY|  61156.0|
|   NH|  20474.0|
|   MI| 366486.0|
|   NV| 474208.0|
|   WI| 152311.0|
|   ID|  22932.0|
|   CA|1891919.0|
|   CT|  54662.0|
|   NE|  59376.0|
|   MT|  19271.0|
+-----+---------+
only showing top 20 rows



For more information, please refer to:
* [Spark SQL, DataFrames and Datasets Guide](http://spark.apache.org/docs/latest/sql-programming-guide.html#sql)
* [PySpark SQL Module: DataFrame](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame)
* [PySpark SQL Functions Module](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions)